In [1]:
%matplotlib inline

In [2]:
import nibabel as nib
import numpy as np
import pandas as pd
import os.path as op

/Users/tpawlowski/workspace/dokstud/mri_pipeline/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
import AFQ.registration as afr

In [4]:
from nilearn.input_data import NiftiMasker

In [5]:
func_filename = op.join(op.expanduser('~'), 'data', 'rtfmri', 'func_0000.nii.gz')

In [6]:
func_img = nib.load(func_filename)  # Nifti1Image

In [7]:
func_data = func_img.get_data()
print("func_data has shape: {}".format(func_data.shape))

func_data has shape: (64, 64, 30, 132)


In [8]:
conditions = pd.read_csv(op.join(op.expanduser('~'), 'data', 'rtfmri', 'timingFiles', 'fullRunLabels.txt'), header=None)
print("all conditions type: {}, shape: {}, preview: {}".format(type(conditions), conditions.shape, conditions.values.tolist()))

all conditions type: <class 'pandas.core.frame.DataFrame'>, shape: (132, 1), preview: [['Fix'], ['Fix'], ['Rest'], ['Rest'], ['Rest'], ['Rest'], ['Rest'], ['Rest'], ['Rest'], ['Rest'], ['Active'], ['Active'], ['Active'], ['Active'], ['Active'], ['Active'], ['Active'], ['Active'], ['Rest'], ['Rest'], ['Rest'], ['Rest'], ['Rest'], ['Rest'], ['Rest'], ['Rest'], ['Active'], ['Active'], ['Active'], ['Active'], ['Active'], ['Active'], ['Active'], ['Active'], ['Rest'], ['Rest'], ['Rest'], ['Rest'], ['Rest'], ['Rest'], ['Rest'], ['Rest'], ['Active'], ['Active'], ['Active'], ['Active'], ['Active'], ['Active'], ['Active'], ['Active'], ['Rest'], ['Rest'], ['Rest'], ['Rest'], ['Rest'], ['Rest'], ['Rest'], ['Rest'], ['Active'], ['Active'], ['Active'], ['Active'], ['Active'], ['Active'], ['Active'], ['Active'], ['Rest'], ['Rest'], ['Rest'], ['Rest'], ['Rest'], ['Rest'], ['Rest'], ['Rest'], ['Active'], ['Active'], ['Active'], ['Active'], ['Active'], ['Active'], ['Active'], ['Active'], ['Rest'], ['Res

In [9]:
condition_mask = conditions[0].isin(['Rest', 'Active'])
conditions = conditions[condition_mask]
func_data = func_data[..., condition_mask]
print("mask: {}, new cond: {}, new shape: {}".format(condition_mask.values.tolist(), conditions.values.tolist(), func_data.shape))

mask: [False, False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, False], new cond: [['Rest'], ['Rest'], ['Rest'], ['Rest'], ['Rest'], ['Rest'], ['Rest'], ['Rest'], ['Active'], ['Active'], ['Active'], ['Active'], ['Active'], ['Active'], ['Active'], ['Active'], ['Rest'],

In [10]:
reference_data = func_data[..., 0]  # changed from func_img.dataobj[..., 0], becaues this frame was filtered out.

print("data: {}".format(reference_data.shape))
print("shape: {}".format(func_img.dataobj.shape))

data: (64, 64, 30)
shape: (64, 64, 30, 132)


In [11]:
# Here we define our ML pipeline: feature selection using ANOVA followed by an SVM classifier:

# Define the dimension reduction to be used.
# Here we use a classical univariate feature selection based on F-test,
# namely Anova. When doing full-brain analysis, it is better to use
# SelectPercentile, keeping 5% of voxels
# (because it is independent of the resolution of the data).
from sklearn.feature_selection import SelectPercentile, f_classif
feature_selection = SelectPercentile(f_classif, percentile=5)

from sklearn.svm import SVC
svc = SVC(kernel='linear')

from sklearn.pipeline import Pipeline
anova_svc = Pipeline([('anova', feature_selection), ('svc', svc)])

# We'll use point by point accuracy score: 
from sklearn.metrics import accuracy_score

In [12]:
fit_data = func_data[..., 0][... , None]

In [13]:
import sys
class Mute():
    def __enter__(self):
        self.old_stdout = sys.stdout
        self.old_stderr = sys.stderr
        sys.stdout = self
        sys.stderr = self
        return self
    
    def __exit__(self, type, value, traceback):
        sys.stdout = self.old_stdout
        sys.stderr = self.old_stderr
        
    def write(self, text):
        pass


In [14]:
accuracies = []
affine = func_img.affine

for idx in range(1, func_data.shape[-1]):
    print("Fit data shape {0}".format(fit_data.shape))
    
    print("Incoming data: ", idx)
    this_data = func_data[..., idx]

    print("Register incoming data to the reference image")
    with Mute() as m:
        this_data, _ = afr.affine_registration(this_data,
                                           reference_data)
    print("Create full series for this iteration")
    fit_data = np.concatenate([fit_data, this_data[..., None]], axis=-1)
    
    if idx > 10:
        mask_data = np.zeros(this_data.shape)
        mask_data[this_data > 0] = 1
        masker = NiftiMasker(mask_img=nib.Nifti1Image(mask_data, affine), #smoothing_fwhm=4,
                             standardize=True, memory_level=1)

        with Mute() as m:
            X = masker.fit_transform(nib.Nifti1Image(fit_data, affine))
            anova_svc.fit(X[:idx], conditions[:idx])
            y_pred = anova_svc.predict(X[idx].reshape(1, -1))
            accuracies.append(accuracy_score(y_pred, conditions.iloc[idx]))
        print("Accurracy: {}".format(accuracies[-1]))

Fit data shape (64, 64, 30, 1)
Incoming data:  1
Register incoming data to the reference image
Create full series for this iteration
Fit data shape (64, 64, 30, 2)
Incoming data:  2
Register incoming data to the reference image
Create full series for this iteration
Fit data shape (64, 64, 30, 3)
Incoming data:  3
Register incoming data to the reference image
Create full series for this iteration
Fit data shape (64, 64, 30, 4)
Incoming data:  4
Register incoming data to the reference image
Create full series for this iteration
Fit data shape (64, 64, 30, 5)
Incoming data:  5
Register incoming data to the reference image
Create full series for this iteration
Fit data shape (64, 64, 30, 6)
Incoming data:  6
Register incoming data to the reference image
Create full series for this iteration
Fit data shape (64, 64, 30, 7)
Incoming data:  7
Register incoming data to the reference image
Create full series for this iteration
Fit data shape (64, 64, 30, 8)
Incoming data:  8
Register incoming da

Accurracy: 0.0
Fit data shape (64, 64, 30, 57)
Incoming data:  57
Register incoming data to the reference image
Create full series for this iteration
Accurracy: 0.0
Fit data shape (64, 64, 30, 58)
Incoming data:  58
Register incoming data to the reference image
Create full series for this iteration
Accurracy: 1.0
Fit data shape (64, 64, 30, 59)
Incoming data:  59
Register incoming data to the reference image
Create full series for this iteration
Accurracy: 1.0
Fit data shape (64, 64, 30, 60)
Incoming data:  60
Register incoming data to the reference image
Create full series for this iteration
Accurracy: 1.0
Fit data shape (64, 64, 30, 61)
Incoming data:  61
Register incoming data to the reference image
Create full series for this iteration
Accurracy: 1.0
Fit data shape (64, 64, 30, 62)
Incoming data:  62
Register incoming data to the reference image
Create full series for this iteration
Accurracy: 1.0
Fit data shape (64, 64, 30, 63)
Incoming data:  63
Register incoming data to the refe

Create full series for this iteration
Accurracy: 1.0
Fit data shape (64, 64, 30, 112)
Incoming data:  112
Register incoming data to the reference image
Create full series for this iteration
Accurracy: 0.0
Fit data shape (64, 64, 30, 113)
Incoming data:  113
Register incoming data to the reference image
Create full series for this iteration
Accurracy: 0.0
Fit data shape (64, 64, 30, 114)
Incoming data:  114
Register incoming data to the reference image
Create full series for this iteration
Accurracy: 0.0
Fit data shape (64, 64, 30, 115)
Incoming data:  115
Register incoming data to the reference image
Create full series for this iteration
Accurracy: 1.0
Fit data shape (64, 64, 30, 116)
Incoming data:  116
Register incoming data to the reference image
Create full series for this iteration
Accurracy: 0.0
Fit data shape (64, 64, 30, 117)
Incoming data:  117
Register incoming data to the reference image
Create full series for this iteration
Accurracy: 1.0
Fit data shape (64, 64, 30, 118)
In